**AULA 3 - VALIDAÇÃO DE DADOS COM PYDANTIC**

**Aqui está uma introdução ao pydantic de como podemos fazer as validações de dados com ele**

In [151]:
#Importando as funções necessárias das bibliotecas
from enum import auto, IntFlag
from typing import Any
from pydantic import BaseModel, EmailStr, Field, SecretStr, ValidationError

In [152]:
"""Criamos uma classe que é do tipo enum e pode ser tanto um autor,
quanto um editor, quanto um desenvolvedor ou um administrador"""
 
class Role(IntFlag):
    Author = auto()
    Editor = auto()
    Developer = auto()
    Admin = Author | Editor | Developer

In [153]:
class User(BaseModel): #Criamos uma classe pro usuário do tipo BaseModel
    name: str = Field(examples=["Raphael"]) #O nome do usuário
    #O email do usuário
    email: EmailStr = Field(
        examples="example@gmail.com", #Exemplo de email
        description="The email adress of the user", #Descrição
        frozen=True, #O email não pode ser alterado
    )
    #A senha do usuário (SecretStr é uma string que não pode ser lida diretamente)
    password: SecretStr = Field(
        examples=["Password123"],
        description="The password of the user",
    )
    #O papel do usuário
    role: Role = Field(
        default=None,
        description="The role of the user"
        )

In [154]:
#Função para validar o usuário
def validate(data: dict[str, Any]) -> None:
    try: #Tentamos validar o usuário
        user = User.model_validate(data) #Validamos o usuário
        print(user) #Printamos o usuário
    except ValidationError as e: #Se houver um erro de validação
        print("User is invalid") #Printamos que o usuário é inválido
        for error in e.errors(): #Para cada erro na lista de erros
            print(error) #Printamos o erro

In [155]:
#Função principal
def main() -> None:
    #Dados bons 
    good_data = {
        "name": "Raphael",
        "email": "example@gmail.com",
        "password": "Password123"
    }
    #Dados ruins
    bad_data = {
        "email": "<bad_data>",
        "password": "<bad_data>"
    }

    validate(good_data) #Validamos os dados bons
    validate(bad_data) #Validamos os dados ruins

In [156]:
#Este é o bloco principal do código
if __name__ == "__main__":
    #Chamamos a função principal do código
    # Se este script for executado diretamente (e não importado), execute a função main()
    main()

name='Raphael' email='example@gmail.com' password=SecretStr('**********') role=None
User is invalid
{'type': 'missing', 'loc': ('name',), 'msg': 'Field required', 'input': {'email': '<bad_data>', 'password': '<bad_data>'}, 'url': 'https://errors.pydantic.dev/2.10/v/missing'}
{'type': 'value_error', 'loc': ('email',), 'msg': 'value is not a valid email address: An email address must have an @-sign.', 'input': '<bad_data>', 'ctx': {'reason': 'An email address must have an @-sign.'}}


**Validação Personalizada**

In [157]:
import enum
import hashlib
import re
import hashlib

In [158]:
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator,
    SecretStr,
    ValidationError,
)

In [159]:
#Usando regex para validação de senhas e nomes de usuários
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z](?=.*\d).{8,}$)")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

In [160]:
class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8


In [161]:
class User(BaseModel):
    name: str = Field(examples=["Arjan"])
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )
    role: Role = Field(
        default=None, description="The role of the user", examples=[1, 2, 4, 8]
    )

    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str:
        if not VALID_NAME_REGEX.match(v):
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}
        try:
            return op[type(v)](v)
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    @model_validator(mode="before")
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

In [162]:
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("User is invalid:")
        print(e)

In [163]:
def main() -> None:
    test_data = dict(
        good_data={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Admin",
        },
        bad_role={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Programmer",
        },
        bad_data={
            "name": "Arjan",
            "email": "bad email",
            "password": "bad password",
        },
        bad_name={
            "name": "Arjan<-_->",
            "email": "example@arjancodes.com",
            "password": "Password123",
        },
        duplicate={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Arjan123",
        },
        missing_data={
            "email": "<bad data>",
            "password": "<bad data>",
        },
    )

    for example_name, data in test_data.items():
        print(example_name)
        validate(data)
        print()

In [164]:
if __name__ == "__main__":
    main()

good_data
name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=<Role.Admin: 4>

bad_role
User is invalid:
1 validation error for User
role
  Value error, Role is invalid, please use one of the following: Author, Editor, Admin, SuperAdmin [type=value_error, input_value='Programmer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

bad_data
User is invalid:
1 validation error for User
  Value error, Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number [type=value_error, input_value={'name': 'Arjan', 'email'...ssword': 'bad password'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

bad_name
User is invalid:
1 validation error for User
name
  Value error, Name is invalid, must contain only letters and be at least 2 characters long [type=value_error, input_value='Arjan<-_->', input_type=str]
    For further information visit https://

**Serialização de dados personalizada**

In [165]:
import enum
import hashlib
import re
from typing import Any, Self
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_serializer,
    field_validator,
    model_serializer,
    model_validator,
    SecretStr,
    )

In [166]:
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

In [167]:
# Definição da enumeração de papéis de usuário
class Role(enum.IntFlag):
    User = 0
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

In [168]:
class User(BaseModel):
    """
    Modelo de usuário baseado no Pydantic, usado para validação e serialização de dados.
    """

    # Campo "name", que deve ser uma string com um exemplo de valor "Example".
    name: str = Field(examples=["Example"])

    # Campo "email", que deve ser um email válido. 
    # Ele inclui um exemplo, uma descrição e é imutável (frozen=True).
    email: EmailStr = Field(
        examples=["user@gmail.com"],
        description="The email address of the user",
        frozen=True,
    )

    # Campo "password", que deve ser uma string secreta.
    # Ele inclui um exemplo e uma descrição, além de ser excluído da saída por padrão.
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user", exclude=True
    )

    # Campo "role", que representa o papel do usuário na aplicação.
    # O valor padrão é 0, e a validação do padrão está ativada.
    role: Role = Field(
        description="The role of the user",
        examples=[1, 2, 4, 8],  # Exemplos de valores possíveis
        default=0,  # Valor padrão
        validate_default=True,  # Ativa a validação para o valor padrão
    )

    # Validador para o campo "name"
    @field_validator("name")
    def validate_name(cls, v: str) -> str:
        """
        Valida se o nome contém apenas letras e tem pelo menos 2 caracteres.
        """
        if not VALID_NAME_REGEX.match(v):  # Verifica se o nome segue o regex definido
            raise ValueError(
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v

    # Validador para o campo "role" antes da conversão do modelo
    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        """
        Converte diferentes tipos de entrada para a enumeração Role.
        Aceita inteiros, strings e instâncias da própria enumeração.
        """
        op = {int: lambda x: Role(x), str: lambda x: Role[x], Role: lambda x: x}  # Mapeia os tipos de entrada
        try:
            return op[type(v)](v)  # Retorna o valor convertido para Role
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    # Validador para o modelo antes da criação da instância
    @model_validator(mode="before")
    @classmethod
    def validate_user_pre(cls, v: dict[str, Any]) -> dict[str, Any]:
        """
        Valida os dados do usuário antes de serem processados:
        - Garante que 'name' e 'password' estejam presentes.
        - Evita que a senha contenha o nome do usuário.
        - Valida se a senha segue os requisitos de segurança.
        - Faz o hash da senha antes de armazená-la.
        """
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")

        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")

        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )

        # Faz o hash da senha usando SHA-256 antes de armazená-la
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

    # Validador para o modelo depois da criação da instância
    @model_validator(mode="after")
    def validate_user_post(self, v: Any) -> Self:
        """
        Valida regras após a criação do usuário:
        - Somente o usuário chamado "Raphael" pode ser um administrador.
        """
        if self.role == Role.Admin and self.name != "Raphael":
            raise ValueError("Only Raphael can be an admin")
        return self

    # Serializador para o campo "role", garantindo que ele seja representado como string no JSON
    @field_serializer("role", when_used="json")
    @classmethod
    def serialize_role(cls, v) -> str:
        """
        Serializa o papel do usuário como uma string em vez de um número.
        """
        return v.name

    # Serializador personalizado para a classe User
    @model_serializer(mode="wrap", when_used="json")
    def serialize_user(self, serializer, info) -> dict[str, Any]:
        """
        Define a forma como o modelo User será serializado:
        - Se nenhum campo for explicitamente incluído/excluído, retorna apenas 'name' e 'role'.
        - Caso contrário, usa a serialização padrão.
        """
        if not info.include and not info.exclude:
            return {"name": self.name, "role": self.role.name}
        return serializer(self)

In [169]:
def main() -> None:
    """
    Função principal que cria um dicionário de dados do usuário, 
    valida-o com o modelo `User` e exibe diferentes formatos de serialização.
    """

    # Dicionário contendo os dados do usuário
    data = {
        "name": "Raphael",
        "email": "example@gmail.com",
        "password": "Password123",
        "role": "Admin",
    }

    # Valida e cria uma instância do modelo User a partir do dicionário `data`
    user = User.model_validate(data)

    # Se a validação for bem-sucedida e a instância `user` for criada
    if user:
        # Serializa o objeto `user` para um dicionário Python
        print(
            "The serializer that returns a dict:",  # Mensagem de saída
            user.model_dump(),  # Converte o modelo `User` para um dicionário
            sep="\n",  # Define o separador como uma nova linha
            end="\n\n",  # Adiciona duas quebras de linha ao final
        )

        # Serializa o objeto `user` para um JSON string
        print(
            "The serializer that returns a JSON string:",
            user.model_dump(mode="json"),  # Converte o modelo `User` para um JSON string
            sep="\n",
            end="\n\n",
        )

        # Serializa para JSON, mas exclui o campo "role"
        print(
            "The serializer that returns a json string, excluding the role:",
            user.model_dump(exclude=["role"], mode="json"),  # Exclui "role" da saída JSON
            sep="\n",
            end="\n\n",
        )

        # Converte o objeto `user` diretamente para um dicionário
        print(
            "The serializer that encodes all values to a dict:",
            dict(user),  # Converte `user` para um dicionário Python
            sep="\n",
        )


In [ ]:
#Execução do código
if __name__ == "__main__":
    main()

The serializer that returns a dict:
{'name': 'Raphael', 'email': 'example@gmail.com', 'role': <Role.Admin: 4>}

The serializer that returns a JSON string:
{'name': 'Raphael', 'role': 'Admin'}

The serializer that returns a json string, excluding the role:
{'name': 'Raphael', 'email': 'example@gmail.com'}

The serializer that encodes all values to a dict:
{'name': 'Raphael', 'email': 'example@gmail.com', 'password': SecretStr('**********'), 'role': <Role.Admin: 4>}


**Integração com FastAPI**